In [79]:
import random
import gurobipy as gp
from gurobipy import GRB
import copy
import numpy as np

n=10
def generate_data(n):
    # Initializing an empty adjacency matrix with zeros
    W = [[0] * n for _ in range(n)]

    # Generate random edge weights (assuming non-negative weights)
    for i in range(n):
        for j in range(i+1, n):
            weight = random.randint(0,10)
            # weight = random.uniform(0, 10)  # Adjust the range as needed
            # Ensure that edges are undirected by setting W[i][j] = W[j][i] = weight
            W[i][j] = W[j][i] = weight
    return W

In [80]:
def BP(W,n):
    # Create a new model
    m = gp.Model("mip1")
    # Create variable

    x = {}
    x = m.addVars(range(n),vtype = "B", name="x")
    y = m.addVars(range(n),range(n),vtype = "B", name="y")

    m.setObjective(sum(W[i][j]*y[i,j] for i in range(n) for j in range(n)), GRB.MAXIMIZE)
    m.addConstrs(y[i,j]<=(x[i]+x[j]) for i in range(n) for j in range(n))
    m.addConstrs(y[i,j]+x[i]+x[j]<=2 for i in range(n) for j in range(n))
    m.optimize()
    y_result=[]

    for v in x.values():
        if v.X == 0 or v.X==-0:
            y_result.append(0)
        else:
            y_result.append(1)
    return y_result

In [81]:
def summing_W(W: list):
    sum_W = []
    for i in range(len(W)):
        value = sum(W[i])
        sum_W.append(value)
    temp_W = W.copy()
    temp_W.append(sum_W.copy())

    for i in range(len(temp_W)):
        if i==len(temp_W)-1:
            temp_W[i].insert(len(temp_W),0)
        else:
            temp_W[i].insert(len(temp_W),sum_W[i])
    return  sum_W, temp_W

In [82]:
def get_data(k):
    xdata1_l = []
    xdata2_l = []
    ydata1_l = []
    for i in range(k):
        data = generate_data(n)
        result = BP(data, n)
        sum_data, matrix_data = summing_W(data)
        xdata1_l.append(sum_data)
        xdata2_l.append(matrix_data)
        ydata1_l.append(result)

    ydata2_l = copy.deepcopy(ydata1_l)
    for i in ydata2_l:
        i.append(-1)

    xdata1 = np.array(xdata1_l)
    xdata2 = np.array(xdata2_l)
    ydata1 = np.array(ydata1_l)
    ydata2 = np.array(ydata2_l)
    return xdata1, xdata2, ydata1, ydata2

In [83]:
k = 1000
xtrain1, xtrain2, ytrain1, ytrain2 = get_data(k)

k = 200
xtest1, xtest2, ytest1, ytest2 = get_data(k)

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 200 rows, 110 columns and 580 nonzeros
Model fingerprint: 0x665d10d0
Variable types: 0 continuous, 110 integer (110 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 120 rows and 60 columns
Presolve time: 0.00s
Presolved: 80 rows, 50 columns, 240 nonzeros
Variable types: 0 continuous, 50 integer (50 binary)
Found heuristic solution: objective 150.0000000

Root relaxation: objective 4.520000e+02, 16 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd  


    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  438.00000    0   10  158.00000  438.00000   177%     -    0s
H    0     0                     250.0000000  438.00000  75.2%     -    0s
H    0     0                     254.0000000  381.00000  50.0%     -    0s
H    0     0                     268.0000000  316.01835  17.9%     -    0s
H    0     0                     282.0000000  316.01835  12.1%     -    0s
     0     0  316.01835    0   49  282.00000  316.01835  12.1%     -    0s
H    0     0                     294.0000000  316.01835  7.49%     -    0s
*    0     0               0     298.0000000  298.00000  0.00%     -    0s

Cutting planes:
  MIR: 12
  Zero half: 1
  RLT: 27

Explored 1 nodes (153 simplex iterations) in 0.09 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 8: 298 294 282 ... -0

Optimal solution found (tolerance

In [ ]:
print(ytrain1)

[[0 1 0 ... 0 1 1]
 [1 0 1 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 ...
 [1 1 1 ... 1 1 0]
 [0 0 1 ... 0 1 1]
 [1 0 1 ... 0 0 1]]


In [ ]:
print(ytest1)

[[0 0 0 ... 1 0 0]
 [0 0 1 ... 0 1 1]
 [0 1 0 ... 1 1 1]
 ...
 [1 0 0 ... 0 1 1]
 [0 0 1 ... 0 1 1]
 [1 0 1 ... 1 0 1]]


In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score

def build_binary_model(xtrain, ytrain, xtest, ytest, nrlayers, nrnodes, rate, k):
    # This function builds the model with nrlayes layers and nrnodes nodes in each layer
    # nrlayers is a number and nrnodes is a vector specifying how many nodes each layer should contain
    model = keras.Sequential()
    numclasses = k

    # add the layers
    for i in range(nrlayers + 1):
        if (i == 0):
            # input layer
            model.add(keras.Input(shape=input_shape))
        else:
            # hidden layers
            nrnode = nrnodes[i - 1]
            model.add(keras.layers.Dense(nrnode, activation="sigmoid"))

    model.add(keras.layers.Dense(numclasses, activation="sigmoid"))

    # give summary of model
    model.summary()

    # train model
    batch_size = 32 # Based on SGD --> 1 gradient is calculated using 32 data points every time
    #so in each epoch, training_size/batch_size many gradient calculations and weight updates
    epochs = 10 # How many times to go through the data to complete the SGD

    opt = keras.optimizers.SGD(learning_rate=rate)
    model.compile(loss="binary_crossentropy",
                  optimizer=opt,
                  metrics=["accuracy"])

    # fit the model, note that keras takes care of the train set/validation set
    model.fit(xtrain, ytrain, batch_size=batch_size, epochs=epochs, validation_split=0.1)

    # get results
    score = model.evaluate(xtest, ytest, verbose=1)
    print("Test loss:", score[0])
    print("Test accuracy:", score[1])
    return model

In [ ]:
print(xtrain1.shape)
print(xtrain2.shape)
print(ytrain1.shape)
print(ytrain2.shape)
print(xtest1.shape)
print(xtest2.shape)
print(ytest1.shape)
print(ytest2.shape)

(1000, 10)
(1000, 11, 11)
(1000, 10)
(1000, 11)
(200, 10)
(200, 11, 11)
(200, 10)
(200, 11)


In [ ]:
input_shape = (n, )
models1 = build_binary_model(xtrain1, ytrain1, xtest1, ytest1, 2, [100, 50], 0.02, n)


Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_45 (Dense)            (None, 100)               1100      
                                                                 
 dense_46 (Dense)            (None, 50)                5050      
                                                                 
 dense_47 (Dense)            (None, 10)                510       
                                                                 
Total params: 6660 (26.02 KB)
Trainable params: 6660 (26.02 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Epoch 1/10
29/29 [==============================] - 1s 16ms/step - loss: 0.7329 - accuracy: 0.0000e+00 - val_loss: 0.7254 - val_accuracy: 0.0000e+00
Epoch 2/10
29/29 [==============================] - 0s 5ms/step - loss: 0.7154 - accuracy: 0.0000e+00 - val_loss: 0.7124 - val_accuracy: 0.0000e+00
Epoch 3/10
29/29 [==============================] - 0s 5ms/step - loss: 0.7060 - accuracy: 0.0000e+00 - val_loss: 0.7050 - val_accuracy: 0.0000e+00
Epoch 4/10
29/29 [==============================] - 0s 6ms/step - loss: 0.7006 - accuracy: 0.0000e+00 - val_loss: 0.7006 - val_accuracy: 0.0000e+00
Epoch 5/10
29/29 [==============================] - 0s 6ms/step - loss: 0.6975 - accuracy: 0.0000e+00 - val_loss: 0.6978 - val_accuracy: 0.0000e+00
Epoch 6/10
29/29 [==============================] - 0s 7ms/step - loss: 0.6955 - accuracy: 0.0000e+00 - val_loss: 0.6963 - val_accuracy: 0.0000e+00
Epoch 7/10
29/29 [==============================] - 0s 6ms/step - loss: 0.6944 - accuracy: 0.0022 - val_loss: 0

In [ ]:
import math

#Return the value of 9 raised to the power of 3


xtrain2=np.reshape(xtrain2,[1000, int(math.pow(n+1,2))])
xtest2=np.reshape(xtest2,[200, int(math.pow(n+1,2))])

input_shape = (int(math.pow(n+1,2)), )
models2 = build_binary_model(xtrain2, ytrain2, xtest2, ytest2, 2, [100, 50], 0.02, n+1)


Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_48 (Dense)            (None, 100)               12200     
                                                                 
 dense_49 (Dense)            (None, 50)                5050      
                                                                 
 dense_50 (Dense)            (None, 11)                561       
                                                                 
Total params: 17811 (69.57 KB)
Trainable params: 17811 (69.57 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Epoch 1/10
29/29 [==============================] - 1s 19ms/step - loss: 0.5015 - accuracy: 0.0056 - val_loss: 0.4194 - val_accuracy: 0.0100
Epoch 2/10
29/29 [==============================] - 0s 11ms/step - loss: 0.3560 - accuracy: 0.0011 - val_loss: 0.2858 - val_accuracy: 0.0100
Epoch 3/10
29/29 [==============================] - 0s 6ms/step - loss: 0.2283 - accuracy: 0.0078 - val_loss: 0.1598 - val_accuracy: 0.0400
Epoch 4/10
29/29 [==============================] - 0s 7ms/step - loss: 0.1003 - accuracy: 0.0400 - val_loss: 0.0293 - val_accuracy: 0.0100
Epoch 5/10
29/29 [==============================] - 0s 12ms/step - loss: -0.0330 - accuracy: 0.0056 - val_loss: -0.1084 - val_accuracy: 0.0100
Epoch 6/10
29/29 [==============================] - 0s 13ms/step - loss: -0.1751 - accuracy: 0.0078 - val_loss: -0.2573 - val_accuracy: 0.0000e+00
Epoch 7/10
29/29 [==============================] - 0s 13ms/step - loss: -0.3309 - accuracy: 0.0044 - val_loss: -0.4184 - val_accuracy: 0.0000e+00
E